#### Find out top 6 locations with the most hourly minimum temperature of 50,000 records (approx. 1/2 of the db)

```sql
select station_id, count(station_id) from air_temp where
airtemp_id in (
select airtemp_id from air_temp
order by temperature,station_id
limit 50000)
group by 1
order by 2 desc
```
> 1st to 6th: S104, S121, S50, S111, S44, S109


#### Check average temperature to see whether the top 6 stations are the same or not
```sql
select station_id, round(avg(temperature),1) from air_temp
join station
using(station_id) where
airtemp_id in (
select airtemp_id from air_temp
order by temperature,station_id
limit 50000)
group by 1
order by 2
```
> S44 & S121: 26.4°C; 
> S109, S50, S104: 26.5°C

In [14]:
# Making a map using the folium module
import folium
temperature_map = folium.Map()

# Top 6 locations that have most hourly records of minimum temperature of sample size of 50000  
stations = [
    {'loc': [1.3764, 103.8492], 'label': 'Ang Mo Kio: 26.5°C'},
    {'loc': [1.3337, 103.7768], 'label': 'Clementi: 26.5°C'},
    {'loc': [1.34583, 103.68166], 'label': 'Nanyang Avenue: 26.4°C'},
    {'loc': [1.37288, 103.72244], 'label': 'Choa Chu Kang: 26.4°C'},
    {'loc': [1.31055, 103.8365], 'label': 'Scotts Road: 26.5°C'},
    {'loc': [1.44387, 103.78538], 'label': 'Woodlands: 26.5°C'}] 

# Adding markers to the map
for station in stations:
    marker = folium.Marker(location=station['loc'], popup=station['label'], icon=folium.Icon(icon='cloud')).add_to(temperature_map)
    marker.add_to(temperature_map)


temperature_map.fit_bounds([(1.31055, 103.68166),(1.44387, 103.8492)])
    
temperature_map


#### After shortlisting the top 6 coolest location, check humidity by generating the humidity rank of all stations from low to high

```sql
with t1 as
(select station_id, round(avg(humidity_readings),1) as average_humidity, rank() over (order by round(avg(humidity_readings),1))
from humidity
group by 1
order by 2)
select t1.station_id, station_name, t1.average_humidity, t1.rank
from t1
join station
using(station_id)
where t1.station_id in ('S104','S121','S50','S111','S44','S109')
```
Rank of the 6 stations are as displayed on the map


In [12]:
# Making a map using the folium module
import folium
from folium.features import DivIcon
humidity_map = folium.Map()

# Humidity readings and station ranking order by lowest humidity 
stations = [
    {'loc': [1.3764, 103.8492], 'label': 'Ang Mo Kio: 75.6', 'rank':'3th Less Humid'},
    {'loc': [1.3337, 103.7768], 'label': 'Clementi: 81.9', 'rank':'13th'},
    {'loc': [1.34583, 103.68166], 'label': 'Nanyang Avenue: 81.0', 'rank':'12th'},
    {'loc': [1.37288, 103.72244], 'label': 'Choa Chu Kang: 83.9', 'rank':'15th'},
    {'loc': [1.31055, 103.8365], 'label': 'Scotts Road: 77.2', 'rank':'7th'},
    {'loc': [1.44387, 103.78538], 'label': 'Woodlands: 81.9', 'rank':'13th'}] 

# Adding markers to the map
for station in stations:
    if station['loc']==[1.3764, 103.8492]:
        marker = folium.Marker(location=station['loc'], popup=station['label'], icon=folium.Icon(icon='thumbs-up',color='orange'))
    else:
        marker = folium.Marker(location=station['loc'], popup=station['label'])
    marker2 = folium.Marker(location=station['loc'], icon=DivIcon(icon_size=(250,36), icon_anchor=(0,0),html='<div style="font-size: 9pt"><p style="display:inline;background-color:white;padding:5px">'+station['rank']+'</p></div>',
)).add_to(humidity_map)
    marker.add_to(humidity_map)


humidity_map.fit_bounds([(1.31055, 103.68166),(1.44387, 103.8492)])
    
humidity_map
